In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import jaccard_similarity_score
from sklearn.preprocessing import MultiLabelBinarizer
import pprint
pp = pprint.PrettyPrinter(indent=4)
from collections import namedtuple

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_similarity_score

Song = namedtuple("Song", ["artist", "title"])
Mood = namedtuple("Mood", ["description", "probability"])

## Load Models

In [3]:
import pickle

trained_af_genre = pickle.load(open('predict_genre_af_AllGenres.pickle', 'rb'))
trained_af_moods = pickle.load(open('predict_mood_audio.pickle', 'rb'))
trained_lyrics_mood = pickle.load(open('trained_lyrics_tfidf.pickle', 'rb'))
database = pd.read_json('songs_have_lyrics.json')

In [4]:
from string import punctuation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer

translator = str.maketrans('', '', punctuation)
stemmer = SnowballStemmer('english')

def clean_text(raw_text):
#   initialize empty clean_words list
    clean_words = []
    raw_text = raw_text.translate(translator)
    raw_text_split = raw_text.split()

    for word in raw_text_split:
        if word not in ENGLISH_STOP_WORDS:
            clean_words.append(stemmer.stem(word))
    return ' '.join(clean_words)

In [5]:
#test song

audio_features = np.array([5,
 0.709932,
 0.231455,
 130.03,
 0.12174099999999999,
 0.036662,
 0.0,
 0,
 4,
 232.46104,
 -5.15,
 0.37439000000000006,
 0.7047289999999999,
 0.5650000000000001,
 0.5650000000000001,
 0.743,
 1.0])
lyrics_features = 'party rock yeah woo let s go party rock is in the house tonight everybody just have a good time and we gon make you loose your mind wooo everybody just have a good time party rock is in the house tonight ooww everybody just have a good time and we gon make you loose your mind yeah we just wanna see you shake that in the club party rock look a pretty girl she on my jock huh non stop when we in the spot booty on the way like she on the block wooo with a drink i gots to know tight jeans tattoos cause i m rock n roll half black half white domino gain the money out the door yoo i m runnin through these hoes like drano i got that devilish flow rock n roll no halo we party rock yeah that s the crew that i m reppin on the rise to the stop no led in our zeppelin heeeey party rock is in the house tonight wooo everybody just have a good time hey and we gon make you loose your mind everybody just have a good time let s go party rock is in the house tonight everybody just have a good time and we gon make you loose your mind we just wanna see you shake that everyday i m shuffling shuffling shuffling step up fast be the first girl to make me throw this cash we gettin money don t be mad now stop hating is bad one more shot for us another round another round please fill up my cup don t mess around bitches wanna see you shake it now now you wanna be you re naked now get up get down put your hands up to the sound get up get down put your hands up to the sound get up get down put your hands up to the sound put your hands up to the sound put your hands up to the sound get up get up get up get up get up get up get up get up get up put your hands up to the sound to the sound put your hands up put your hands up put your hands up put your hands up party rock is in the house tonight put your hands up everybody just have a good time put your hands up and we gon make you loose your mind put your hands up everybody just have a good good good time ooohh ooooh put your hands up ooohh ooooh ooh ooh ooohh ooooh put your hands up ooohh ooooh put your hands up shake that everyday i m shu ff ffling put your put your put your put your yeaah put your put your wooo put your put your your hands up put your hands up put your hands up everyday i m shuffling'

In [6]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(database['moods'])
cleaned_lyrics = clean_text(lyrics_features)
moods = trained_lyrics_mood.predict([cleaned_lyrics])

audio_features.reshape((1, -1))

array([[ 5.0000000e+00,  7.0993200e-01,  2.3145500e-01,  1.3003000e+02,
         1.2174100e-01,  3.6662000e-02,  0.0000000e+00,  0.0000000e+00,
         4.0000000e+00,  2.3246104e+02, -5.1500000e+00,  3.7439000e-01,
         7.0472900e-01,  5.6500000e-01,  5.6500000e-01,  7.4300000e-01,
         1.0000000e+00]])

In [7]:
mlb.inverse_transform(moods)

[('celebratory', 'happy', 'rowdy')]

In [55]:
def recommend_similar_songs(audio_features, lyrics_features):
    cleaned_lyrics = clean_text(lyrics_features)
    predicted_genre = trained_af_genre.predict(audio_features.reshape((1, -1)))
    #predicted_moods_AF = trained_af_moods.predict(audio_features.reshape((1, -1)))
    predicted_moods = trained_lyrics.predict([cleaned_lyrics])
    
    # need to compare predicted moods AF and predicted moods on lyrics
    
    # find top 3 moods
    
    # return 
#{   'genre': 'Jazz',
 #   'mood': [   Mood(description='Happy', probability=0.75),
  #              Mood(description='Energetic', probability=0.71),
   #             Mood(description='Celebratory', probability=0.68)],
    #'playlist': [   Song(artist='kanye west', title='i am a god'),
     #               Song(artist='linkin park', title='crawling')]}
    